# Exercises: neural network review/introduction

Exercise on classification using NN. 

Author: Stefano Pagani <stefano.pagani@polimi.it>.

Date: 2024

Course: Mathematical and numerical foundations of scientific machine learning.

Example adapted from:
[1] Brunton, S. L., & Kutz, J. N. (2022).
Data-driven science and engineering: Machine learning,
dynamical systems, and control. Cambridge University Press.
Data from [1].


In [ ]:

# imports

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import io
import os
from sklearn import linear_model
import tensorflow as tf
from tensorflow import keras

from keras import layers
from keras import regularizers
from keras.utils import to_categorical



In [ ]:


# Read dataset
rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = [9, 6]

dogdata_mat = io.loadmat(os.path.join('DATA','dogData.mat'))
catdata_mat = io.loadmat(os.path.join('DATA','catData.mat'))

dog = dogdata_mat['dog']
cat = catdata_mat['cat']

CD = np.concatenate((dog,cat),axis=1)

X_train =np.concatenate((dog[:,:60],cat[:,:60]),axis=1)
X_test = np.concatenate((dog[:,60:80],cat[:,60:80]),axis=1)
y_train = np.repeat(np.array([1,-1]),60)
y_test = np.repeat(np.array([1,-1]),20)




Task 1: visualize some pictures from the training set

In [ ]:


plt.figure()

indp = 33

plt.imshow(np.reshape(X_train[:,indp],(64,64)).T,cmap='gray')
plt.title('Train picture')
plt.pause(1.0)


indp = 101

plt.imshow(np.reshape(X_train[:,indp],(64,64)).T,cmap='gray')
plt.title('Train picture')
plt.pause(1.0)




Task 2: we can formulate the problem as a least-squares problem of minimizing norm(X_trian x-y_train). Suggestion: use the Moore-Penrose Pseudoinverse.

In [ ]:

print(np.shape(np.linalg.pinv(X_train.T)))

# \ell_2 misfit loss: use pinv
A = np.linalg.pinv(X_train.T) @ y_train.T 
test_labels = np.sign( A @ X_test )


print(test_labels)




Task 3: we can formulate the problem as a least-squares problem plus a $\ell_1$ regularization (Lasso). Suggestion: use the command linear_model.Lasso(alpha=0.1).

In [ ]:


# # \ell_2 misfit loss + weight \ell1 regularization: use lasso
lasso = (linear_model.Lasso(alpha=0.1)).fit(X_train.T,y_train)
A_lasso = lasso.coef_
test_labels_lasso = np.sign(A_lasso@X_test)

print(test_labels_lasso)




Task 3: implement a single layer (two neurons) neural network to classify the pictures.

In [ ]:

# Keras NN

from keras.models import Sequential   # importing Sequential model
from keras.layers import Dense        # importing Dense layers

X_tf_train = tf.constant(  np.transpose(X_train)/256, dtype = tf.float32 )
X_tf_test = tf.constant( np.transpose(X_test)/256, dtype = tf.float32 )
# y_tf_train = tf.constant( np.repeat(np.array([1,0]),60) , dtype = tf.float32 )
# y_tf_test = tf.constant( np.repeat(np.array([1,0]),20), dtype = tf.float32 )

y = np.array([[1,0],[0,1]])

y_tf_train = tf.constant( np.repeat(y, [60, 60], axis=0) , dtype = tf.float32 )
y_tf_test = tf.constant( np.repeat(y, [20, 20], axis=0) , dtype = tf.float32 )


In [ ]:

# declaring model
basic_model = Sequential()

# Adding layers to the model
basic_model.add(Dense(units = 2 , activation = 'softmax', input_shape = (64*64,)))#kernel_regularizer = regularizers.l1(0.0025)))

sgd = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=True)
basic_model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])

# training the model
basic_model.fit(X_tf_train, y_tf_train, epochs=250)

# Test, Loss and accuracy
loss_and_metrics = basic_model.evaluate(X_tf_test, y_tf_test)
print('Loss = ',loss_and_metrics[0])
print('Accuracy = ',loss_and_metrics[1])


In [ ]:


fig,axs = plt.subplots(4,1)
axs[0].bar(range(len(test_labels)),y_test)
NNpred = np.sign(np.array(-1.0 +2.0*basic_model(X_tf_test).numpy()))
#NNpred = (tf.argmax(basic_model(X_tf_test),axis=1).numpy())
axs[1].bar(range(len(test_labels)),NNpred[:,0])
axs[1].set_title('Single neuron')
axs[2].bar(range(len(test_labels)),test_labels)
axs[2].set_title('regression')
axs[3].bar(range(len(test_labels)),test_labels_lasso)
axs[3].set_title('lasso')
plt.show()
